In [3]:
import json
import os
import re
from openai import OpenAI # GPT-4o 사용 시
from anthropic import Anthropic
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [ ]:

# # --- 유틸리티 함수 ---
# def sanitize_filename(name):
#     """파일 이름으로 사용하기 어려운 문자를 제거하거나 대체합니다."""
#     if not isinstance(name, str): # 문자열이 아닌 경우 처리
#         name = str(name)
#     name = re.sub(r'[<>:"/\\|?*]', '_', name) # 파일명 금지 문자 대체
#     name = re.sub(r'\s+', '_', name) # 공백을 밑줄로
#     return name[:100] # 파일명 길이 제한 (필요시)


# class RequirementsLoader:
#     def load_from_file(self, filepath):
#         try:
#             with open(filepath, 'r', encoding='utf-8') as f:
#                 data = json.load(f)
#             print(f"요구사항 파일 '{filepath}' 로드 성공.")
#             return data
#         except FileNotFoundError:
#             print(f"오류: 파일 '{filepath}'를 찾을 수 없습니다.")
#             return None
#         except json.JSONDecodeError:
#             print(f"오류: 파일 '{filepath}'가 유효한 JSON 형식이 아닙니다.")
#             return None
#         except Exception as e:
#             print(f"파일 로드 중 예기치 않은 오류 발생: {e}")
#             return None

# class RequirementsAnalyzer:
#     def __init__(self, requirements_data, openai_client=None):
#         self.requirements = requirements_data
#         self.client = openai_client 
#         self.model = "gpt-4o" 
#         self.analysis_cache = {}

#     def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
#         if not self.client:
#             print(f"OpenAI 클라이언트가 없어 GPT 분석을 건너뜁니다 ({cache_key}).")
#             return None
#         if cache_key in self.analysis_cache:
#             return self.analysis_cache[cache_key]
        
#         try:
#             if hasattr(self.client, 'chat') and hasattr(self.client.chat, 'completions'): 
#                  response = self.client.chat.completions.create(
#                     model=self.model,
#                     messages=[
#                         {"role": "system", "content": system_message},
#                         {"role": "user", "content": prompt_text}
#                     ],
#                     temperature=0.2,
#                     max_tokens=2048
#                 )
#                  result = response.choices[0].message.content.strip()
#             else: 
#                 result = "GPT 호출 방식 오류로 분석 결과 없음"
#                 print(f"⚠️ RequirementsAnalyzer._call_gpt: 클라이언트 API 형식이 예상과 다릅니다. ({cache_key})")

#             if not result:
#                 print(f"⚠️ GPT 응답이 비어 있음. 키: {cache_key}")
#                 return None
#             self.analysis_cache[cache_key] = result
#             return result
#         except Exception as e:
#             print(f"❌ GPT 호출 실패 (키: {cache_key}) → {e}")
#             return None

#     def get_feature_specifications(self):
#         feature_specs = []
#         if not self.requirements:
#             print("오류: 분석할 요구사항 데이터가 없습니다.")
#             return feature_specs

#         # "type"이 "기능"인 항목만 필터링 (우선순위/필수 항목 없음)
#         target_reqs = [
#             req for req in self.requirements
#             if req.get("type") == "기능"
#         ]
#         print(f"분석 대상 기능적 요구사항 수: {len(target_reqs)}")

#         for i, req in enumerate(target_reqs):
#             req_id = f"FUNC-{i+1:03}"  # 새로 ID 생성
#             description = req.get("description_name", "제목 없음")
#             detail = req.get("description_content", "") + "\n\n" + req.get("processing_detail", "")
#             actor_guess = "사용자"  # 역할 정보가 명시적으로 없음, 추정 필요

#             feature_specs.append({
#                 "id": req_id,
#                 "description": description.strip(),
#                 "description_detailed": detail.strip(),
#                 "acceptance_criteria": "요구사항 내 명시 없음",  # 명확한 수용 기준 없음
#                 "ui_suggestion_raw": f"'{description}' 기능을 위한 UI 구성 요소 제안",
#                 "actor_suggestion": actor_guess,
#                 "module": req.get("target_task", "미정"),
#                 "priority": req.get("importance", "중"),
#             })

#         print(f"{len(feature_specs)}개의 주요 기능 명세 추출 완료.")
#         return feature_specs

#     def get_system_overview(self):
#         if not self.requirements: 
#             print("오류: 시스템 개요를 파악할 요구사항 데이터가 없습니다.")
#             return "요구사항 데이터 없음"

#         first_req_desc = self.requirements[0].get("description", "상세 설명 없음")
#         num_total_reqs = len(self.requirements)
        
#         sample_descriptions_for_overview = "\n".join([
#             f"- ID:{req.get('id')}, 설명:{req.get('description')}" 
#             for req in self.requirements[:min(10, len(self.requirements))]
#         ])
        
#         prompt = f"""다음은 소프트웨어 요구사항의 일부입니다:
#         {sample_descriptions_for_overview}
#         ---
#         위 요구사항들을 종합하여, 이 시스템의 주요 목적은 무엇이며, 예상되는 주요 사용자 역할(액터)들은 누구인지, 그리고 이 시스템을 대표할 만한 간결한 이름이나 주제가 있다면 무엇인지 요약해주십시오.
#         """
#         overview = self._call_gpt(prompt, "system_overview_summary_v3", "You are a system architect summarizing project requirements.")
        
#         if overview:
#             print(f"시스템 개요 파악 (GPT): {overview[:100]}...")
#             return overview
#         else:
#             fallback_overview = f"총 {num_total_reqs}개의 요구사항을 가진 시스템. 주요 목적은 '{first_req_desc}'와 관련될 것으로 보이며, 다양한 사용자 역할(학습자, 관리자 등)을 지원할 것으로 예상됩니다."
#             print(f"시스템 개요 파악 (Fallback): {fallback_overview[:100]}...")
#             return fallback_overview

# class MockupPlanner:
#     def __init__(self, feature_specs, system_overview, openai_client=None):
#         self.feature_specs = feature_specs
#         self.system_overview = system_overview
#         self.client = openai_client
#         self.analysis_cache = {}

#     def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
#         if not self.client:
#             print(f"OpenAI 클라이언트가 없어 GPT 계획 수립을 건너뜁니다 ({cache_key}).")
#             return None
#         if cache_key in self.analysis_cache:
#             return self.analysis_cache[cache_key]
        
#         try:
#             print(f"GPT 계획 요청 중 (키: {cache_key})...")
#             response = self.client.chat.completions.create(
#                 model="gpt-4o",
#                 messages=[
#                     {"role": "system", "content": system_message},
#                     {"role": "user", "content": prompt_text}
#                 ]
#             )
#             result = response.choices[0].message.content
#             self.analysis_cache[cache_key] = result
#             return result
#         except Exception as e:
#             print(f"GPT API 호출 중 오류 발생 ({cache_key}): {e}")
#             return None

#     def define_pages_and_allocate_features(self):
#         if not self.feature_specs:
#             print("페이지 계획을 위한 기능 명세가 없습니다.")
#             return self._get_fallback_page_plan()

#         features_text_for_gpt = ""
#         for spec in self.feature_specs: 
#             features_text_for_gpt += f"- ID: {spec['id']}\n  기능 설명: {spec['description']}\n  (UI 제안: {spec['ui_suggestion_raw']}, 대상 액터 추정: {spec['actor_suggestion']}, 우선순위: {spec.get('priority', 'N/A')})\n\n"

#         prompt = f"""
#             다음은 구축할 소프트웨어 시스템의 개요와 주요 기능 명세입니다:

#             시스템 개요:
#             {self.system_overview}

#             주요 기능 명세 (ID, 설명, UI 제안, 대상 액터 추정, 우선순위 순):
#             {features_text_for_gpt}

#             ---
#             위 정보를 바탕으로, 이 시스템에 필요한 웹 페이지(화면)들의 목록을 제안해주십시오. 
#             **매우 중요: 위에 제시된 '주요 기능 명세'의 모든 항목이 결과적으로 하나 이상의 페이지에 반드시 할당되어야 합니다. 누락되는 기능이 없도록 각별히 신경 써주십시오.**
#             사용자 경험 흐름(User Flow)과 정보 구조(Information Architecture)를 고려하여, 기능들이 논리적으로 그룹화되고 중복이 최소화되도록 페이지를 구성해주십시오.
#             **"필수" 우선순위 기능을 반드시 포함**하는 페이지들을 우선적으로 고려해주십시오.
#             시스템의 주요 목적(예: 온라인 교육 플랫폼, 스포츠 지원 포털)을 충분히 고려하여 페이지들을 제안해주십시오.

#             각 페이지에 대해 다음 정보를 포함하여 **JSON 형식**으로 응답해주십시오. 
#             결과는 'pages'라는 최상위 키를 가진 딕셔너리이거나, 페이지 정보 딕셔너리들의 리스트 자체일 수 있습니다. 
#             만약 리스트 자체로 응답한다면, 각 요소는 다음 키들을 포함해야 합니다:
#             1.  `page_name`: 페이지의 대표적인 이름 (예: "User_Login", "Learner_Dashboard", "Course_Browse_And_Apply"). 파일명으로 사용하기 좋게 영어와 밑줄로 구성해주십시오.
#             2.  `page_title_ko`: 페이지의 한글 제목 (HTML title 태그 및 화면 표시용).
#             3.  `page_description`: 이 페이지의 주요 목적과 핵심 기능에 대한 간략한 설명.
#             4.  `target_actors`: 이 페이지를 주로 사용할 사용자 역할(들) (리스트 형태, 예: ["학습자"], ["관리자", "운영자"]).
#             5.  `included_feature_ids`: 이 페이지에 포함되어야 할 주요 기능들의 ID (위 기능 명세의 ID들을 참조하여 리스트 형태로, 예: ["FUNC-001", "DATA-003"]). **이 페이지에 할당하기로 결정한 모든 기능의 ID를 빠짐없이 포함해야 합니다.**
#             6.  `key_ui_elements_suggestion`: 이 페이지의 핵심 UI 컴포넌트들에 대한 구체적인 제안 (문자열 형태).

#             만약 제안할 페이지가 없다면 빈 리스트 `[]`를 'pages' 키의 값으로 주거나, 빈 리스트 자체를 응답해주십시오.
#             """

#         print("GPT에 페이지 정의 및 기능 할당 요청...")
#         page_definitions_str = self._call_gpt(prompt, "page_definitions_v5_flexible", 
#                                                 "You are an expert UI/UX designer and information architect. Respond ONLY in valid JSON format. The response can be a JSON object with a 'pages' key containing a list, OR it can be a list of page objects directly.")
        
#         if page_definitions_str:
#             try:
#                 match = re.search(r'```json\s*([\s\S]*?)\s*```', page_definitions_str, re.IGNORECASE)
#                 if match:
#                     json_str_cleaned = match.group(1)
#                 else:
#                     json_str_cleaned = page_definitions_str.strip()
                
#                 parsed_response = json.loads(json_str_cleaned)

#                 pages_list = None
#                 if isinstance(parsed_response, list): 
#                     pages_list = parsed_response
#                     print(f"GPT로부터 {len(pages_list)}개의 페이지 계획 (리스트 직접 반환)을 성공적으로 받았습니다.")
#                 elif isinstance(parsed_response, dict) and "pages" in parsed_response and isinstance(parsed_response.get("pages"), list):
#                     pages_list = parsed_response["pages"]
#                     print(f"GPT로부터 {len(pages_list)}개의 페이지 계획 ('pages' 키 사용)을 성공적으로 받았습니다.")
                
#                 if pages_list is not None: 
#                     if not pages_list: 
#                         print("GPT가 제안한 페이지가 없습니다. 대체 계획을 사용합니다.")
#                         return self._get_fallback_page_plan()
#                     return pages_list
#                 else: 
#                     print(f"GPT 응답이 예상된 형식이 아닙니다. 응답 내용: {parsed_response}")
#                     return self._get_fallback_page_plan()

#             except json.JSONDecodeError as e:
#                 print(f"GPT 페이지 계획 응답 파싱 오류: {e}. 응답 내용:\n{page_definitions_str}")
#                 return self._get_fallback_page_plan()
#             except Exception as e:
#                 print(f"페이지 계획 처리 중 예기치 않은 오류: {e}")
#                 return self._get_fallback_page_plan()
#         else:
#             print("GPT로부터 페이지 계획을 받지 못했습니다.")
#             return self._get_fallback_page_plan()

#     def _get_fallback_page_plan(self):
#         print("대체 페이지 계획 사용...")
#         if not self.feature_specs:
#             return [{
#                 "page_name": "Error_No_Features",
#                 "page_title_ko": "오류 - 기능 정보 없음",
#                 "page_description": "분석할 기능 명세가 없어 페이지를 계획할 수 없습니다.",
#                 "target_actors": ["개발자"],
#                 "included_feature_ids": [],
#                 "key_ui_elements_suggestion": "오류 메시지 표시."
#             }]
        
#         main_page_features = [spec['id'] for spec in self.feature_specs if spec.get('priority') == '필수']
#         if not main_page_features: 
#             main_page_features = [spec['id'] for spec in self.feature_specs[:min(3, len(self.feature_specs))]]
        
#         actors_for_fallback = set()
#         for spec_id in main_page_features:
#             spec = next((s for s in self.feature_specs if s['id'] == spec_id), None)
#             if spec and spec.get('actor_suggestion'):
#                 current_actors = spec['actor_suggestion']
#                 if isinstance(current_actors, str):
#                     actors_for_fallback.update(act.strip() for act in current_actors.split('/'))
#                 elif isinstance(current_actors, list):
#                     actors_for_fallback.update(current_actors)

#         return [{
#             "page_name": "Main_Application_Page_Fallback",
#             "page_title_ko": "주요 애플리케이션 화면 (대체)",
#             "page_description": "시스템의 핵심 기능들을 제공하는 기본 페이지입니다. (GPT 계획 실패로 인한 대체 화면)",
#             "target_actors": list(actors_for_fallback) if actors_for_fallback else ["일반 사용자"],
#             "included_feature_ids": main_page_features,
#             "key_ui_elements_suggestion": "이 페이지는 다음 기능들을 포함합니다: " + ", ".join(main_page_features) + ". 각 기능에 맞는 UI 요소(버튼, 테이블, 폼 등)가 필요합니다."
#         }]
# # END: 이전 답변의 클래스들


# class HtmlGenerator:
#     def __init__(self, openai_client):
#         self.client = openai_client
#         self.analysis_cache = {}

#     def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant.", temperature=0.15): # 기본 temperature 조정
#         # ... (이전과 동일) ...
#         if not self.client:
#             # ...
#             return None
#         if cache_key in self.analysis_cache:
#             # ...
#             return self.analysis_cache[cache_key]
        
#         try:
#             # ... (API 호출 부분) ...
#             print(f"GPT HTML 생성 요청 중 (키: {cache_key})...")
#             response = self.client.chat.completions.create(
#                 model="gpt-4o", # 또는 최신/최고 성능 모델
#                 messages=[
#                     {"role": "system", "content": system_message},
#                     {"role": "user", "content": prompt_text}
#                 ],
#                 temperature=temperature 
#             )
#             result = response.choices[0].message.content
            
#             match = re.search(r'```(html)?\s*([\s\S]*?)\s*```', result, re.IGNORECASE)
#             if match:
#                 html_code = match.group(2).strip()
#             else:
#                 html_code = result.strip()

#             self.analysis_cache[cache_key] = html_code
#             return html_code
#         except Exception as e:
#             # ... (오류 처리) ...
#             print(f"GPT HTML 생성 API 호출 중 오류 발생 ({cache_key}): {e}")
#             return f"\n<!DOCTYPE html>\n<html><head><title>오류</title></head><body><h1>HTML 생성 중 오류 발생</h1><p>키: {cache_key}</p><p>오류 내용: {e}</p></body></html>"


#     def generate_html_for_page_plan(self, page_plan_details, all_feature_specs):
#         # ... (메서드 상단 및 상세 요구사항 변수 준비는 이전 답변과 거의 동일) ...
#         page_title_ko = page_plan_details.get("page_title_ko", "목업 페이지")
#         # ... (기타 변수들: page_name_en, page_description, target_actors, default_placeholder, user_story_str, acceptance_criteria_str, ui_elements_list_str, data_fields_str, layout_guidelines_str, basic_style_guide_str, api_interactions_str, key_ui_elements_suggestion, features_details_for_prompt, is_responsive, detailed_functional_requirements_section_str, detailed_interface_requirements_section_str - 이전 답변에서 복사)
#         # (이하 생략된 변수 준비 코드는 바로 이전 답변을 참고해주세요)
#         # ----- 변수 준비 시작 (이전 답변 내용 일부 복사) -----
#         page_name_en = page_plan_details.get("page_name", "UnknownPage")
#         page_description = page_plan_details.get("page_description", "N/A")
#         target_actors = ", ".join(page_plan_details.get("target_actors", [])) if isinstance(page_plan_details.get("target_actors"), list) else str(page_plan_details.get("target_actors", ""))

#         default_placeholder = "제공되지 않음"
#         user_story_str = page_plan_details.get("user_story", default_placeholder)
#         acceptance_criteria_list = page_plan_details.get("acceptance_criteria", [])
#         acceptance_criteria_str = "\n".join([f"- {ac}" for ac in acceptance_criteria_list]) if acceptance_criteria_list else default_placeholder
        
#         ui_elements_list = page_plan_details.get("ui_elements_needed", [])
#         ui_elements_list_str = "\n".join([f"- {elem}" for elem in ui_elements_list]) if ui_elements_list else default_placeholder
        
#         data_fields_list = page_plan_details.get("data_fields_to_display", [])
#         data_fields_str = "\n".join([f"- {field}" for field in data_fields_list]) if data_fields_list else default_placeholder
        
#         layout_guidelines_str = page_plan_details.get("layout_guidelines", default_placeholder)
#         basic_style_guide_str = page_plan_details.get("basic_style_guide", default_placeholder)
        
#         api_interactions_list = page_plan_details.get("api_interactions", [])
#         api_interactions_str = ""
#         if api_interactions_list:
#             for interaction in api_interactions_list:
#                 api_interactions_str += f"- 요소/기능: {interaction.get('action_description', interaction.get('element_id', 'N/A'))}\n"
#                 api_interactions_str += f"  엔드포인트: {interaction.get('endpoint', 'N/A')}\n"
#                 api_interactions_str += f"  HTTP 메서드: {interaction.get('method', 'N/A')}\n"
#                 if interaction.get('request_fields'):
#                     api_interactions_str += f"  요청 데이터 필드: {', '.join(interaction.get('request_fields'))}\n"
#                 if interaction.get('response_notes'):
#                     api_interactions_str += f"  예상 응답/처리: {interaction.get('response_notes')}\n\n"
#         else:
#             api_interactions_str = "이 페이지와 직접 관련된 주요 API 연동 정보가 명시되지 않음."

#         key_ui_elements_suggestion = page_plan_details.get("key_ui_elements_suggestion", "기본 콘텐츠 영역")
#         included_feature_ids = page_plan_details.get("included_feature_ids", [])
#         features_details_for_prompt = ""
#         if included_feature_ids:
#             for req_id in included_feature_ids:
#                 feature = next((spec for spec in all_feature_specs if spec["id"] == req_id), None)
#                 if feature:
#                     desc = feature.get('description_detailed', feature.get('description', 'N/A'))
#                     acc_crit = feature.get('acceptance_criteria_summary', feature.get('acceptance_criteria', 'N/A'))
#                     features_details_for_prompt += f"- 기능 ID {feature['id']}: {desc}\n  (수용 조건 요약: {acc_crit})\n\n"
#         else:
#             features_details_for_prompt = "이 페이지에 직접 할당된 세부 기능 명세가 없습니다.\n"
#         is_responsive = True

#         if user_story_str == default_placeholder and acceptance_criteria_str == default_placeholder:
#             detailed_functional_requirements_section_str = "(요청 시 제공된 상세 기능 정보 없음)"
#         else:
#             detailed_functional_requirements_section_str = f"""
#         - 사용자 스토리: {user_story_str}
#         - 주요 수용 기준 (Acceptance Criteria):
# {acceptance_criteria_str}"""

#         if (ui_elements_list_str == default_placeholder and
#             data_fields_str == default_placeholder and
#             layout_guidelines_str == default_placeholder and
#             basic_style_guide_str == default_placeholder):
#             detailed_interface_requirements_section_str = "(요청 시 제공된 상세 인터페이스 정보 없음)"
#         else:
#             detailed_interface_requirements_section_str = f"""
#         - 이 페이지에 필요한 주요 UI 요소 목록 (형식: 요소타입:이름:표시텍스트 또는 설명):
# {ui_elements_list_str}
#         - 페이지에 표시되어야 할 주요 데이터 필드 (테이블, 리스트, 카드 등에 해당):
# {data_fields_str}
#         - 기본 레이아웃 가이드라인: {layout_guidelines_str}
#         - 초기 스타일/브랜딩 가이드라인 (제공된 경우): {basic_style_guide_str}"""
#         # ----- 변수 준비 끝 -----

#         prompt = f"""
#         웹 페이지의 HTML 목업 코드를 생성해주십시오. 이 목업은 단순한 와이어프레임을 넘어, **전문 UI 디자이너가 Stitch나 Figma와 같은 전문 도구를 사용하여 제작한 수준의 매우 높은 시각적 완성도와 전문성**을 목표로 합니다. 
#         API 연동을 준비하는 구조를 갖추되, JavaScript 없이 순수 HTML/CSS로 작성됩니다.

#         **목표 컨텍스트:** Figma에서 상세 UI 디자인으로 즉시 활용 가능하며, 이후 백엔드 API와 연동하여 실제 작동하는 애플리케이션으로 개발될 **최소 실행 가능한 기초 자료(MCP)**입니다.
#         **스타일 목표:** **극도로 깔끔하고(immaculate), 정교하며(sophisticated), 현대적인(modern) 미니멀리즘 UI 디자인**을 구현합니다. 모든 디자인 요소는 의도적이어야 하며, 최고 수준의 미적 감각을 반영해야 합니다.

#         **페이지 기본 정보:**
#         - 한글 페이지 제목: "{page_title_ko}"
#         - 페이지 영문명 (내부 참조용): "{page_name_en}"
#         - 페이지 주요 목적: "{page_description}"
#         - 주요 대상 사용자: "{target_actors}"

#         **상세 기능 요구사항:**
#         {detailed_functional_requirements_section_str}

#         **상세 인터페이스 요구사항:**
#         {detailed_interface_requirements_section_str}

#         **주요 API 연동 정보 (이 페이지에서 예상되는):**
#         {api_interactions_str}
#         (주: 이 정보를 바탕으로, HTML 요소에 `data-*` 속성 등을 추가하거나, 주석으로 API 연동을 위한 준비를 해주십시오.)

#         **페이지에 포함되어야 할 핵심 기능 및 UI 요소 제안 (위 상세 요구사항이 우선):**
#         {key_ui_elements_suggestion}

#         **참고할 기타 관련 기능 정보:**
#         {features_details_for_prompt}

#         **HTML 생성 가이드라인 (전문 UI 디자이너 수준):**
#         1.  **완전한 HTML 문서 구조** 및 **필수 Meta 태그**를 포함해주십시오.
#         2.  **시맨틱 HTML & Figma/개발 친화적 구조:**
#             -   HTML5 시맨틱 태그를 최대한 활용하고, 모든 요소는 논리적으로 그룹화되어야 합니다.
#             -   CSS 클래스명은 BEM(Block, Element, Modifier) 방법론이나 유사한 체계적인 명명 규칙을 사용하여 매우 명확하고 재사용 가능하도록 작성해주십시오. (예: `class="card product-card product-card--featured"`)
#             -   **API 연동 준비:** 데이터 표시 영역에는 명확한 `id`를, 인터랙티브 요소에는 `data-action` 등의 `data-*` 속성을 부여하고, 폼에는 각 `input`에 `name` 속성을 명시해주십시오. API 호출 정보는 주석으로 상세히 기술합니다.
#         3.  **인라인 CSS 스타일 (최고 수준의 미니멀 & 모던 디자인):**
#             -   **모든 CSS 스타일은 HTML 코드 내 `<style>` 태그 안에 포함**해주십시오.
#             -   **전반적인 디자인 철학:** "Less is more, but every detail matters." 모든 디자인 결정은 목적이 있어야 하며, 최고의 사용자 경험과 미적 완성도를 추구합니다. 일반적이거나 미숙해 보이는 스타일링은 절대적으로 피해주십시오.
#             -   **정교한 레이아웃(Sophisticated Layouts):**
#                 -   CSS Grid와 Flexbox를 창의적이고 효과적으로 조합하여, 시각적으로 매우 매력적이고 안정적인 페이지 구조를 설계하십시오. 필요시 비대칭 레이아웃이나 복합 그리드를 적용하여 단조로움을 피하고 디자인에 깊이를 더하십시오.
#                 -   사용자의 시선을 자연스럽게 유도하는 명확한 시각적 흐름(visual flow)을 만드십시오. 모든 요소는 의도된 위치에 정렬(alignment)되어야 합니다.
#             -   **세련된 컴포넌트 스타일링(Refined Component Design):**
#                 -   버튼, 폼 요소(입력창, 셀렉트박스, 라디오/체크박스), 카드, 내비게이션, 탭, 아코디언, 모달, 툴팁 등 모든 UI 요소는 극도로 세심한 주의를 기울여 스타일링해야 합니다. 각 요소는 명확한 사용성(affordance)과 미적인 아름다움을 동시에 가져야 합니다.
#                 -   미묘하지만 명확한 `hover`, `focus`, `active`, `disabled` 상태 스타일을 모든 인터랙티브 요소에 일관되게 적용하십시오. (예: `focus` 시 은은한 외곽선 또는 그림자 변화)
#             -   **고급 타이포그래피(Advanced Typography):**
#                 -   정교한 타이포그래피 스케일(typographic scale)과 수직 리듬(vertical rhythm)을 적용하여, 명확한 정보 계층과 뛰어난 가독성을 동시에 달성하십시오.
#                 -   폰트는 극도로 가독성이 높고 현대적인 산세리프 계열(예: Inter, Figtree, 또는 시스템 UI 폰트 스택 `-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif`)을 사용하십시오.
#                 -   다양한 텍스트 요소(H1-H6, 본문, 캡션, 인용구 등)에 맞는 정밀한 폰트 크기, 굵기(font-weight), 자간(letter-spacing, 예: -0.01em ~ -0.03em), 행간(line-height, 예: 1.5 ~ 1.8) 설정을 적용하십시오.
#             -   **의도적인 색상 팔레트(Intentional Color Palette):**
#                 -   전문가가 설계한 듯한, 극도로 제한적이고 조화로운 색상 팔레트를 구성하십시오. 주로 밝고 깨끗한 배경(예: `#FFFFFF`, `#F7F7F7`) 위에 매우 높은 명암비를 가지는 텍스트 색상(예: `#111111`, `#333333`)을 사용합니다.
#                 -   단 하나의 주요 액센트 컬러(예: 세련된 파란색 `#0070C9` 또는 제공된 브랜딩 가이드의 핵심 색상)를 선택하고, 이를 클릭 유도 버튼이나 가장 중요한 하이라이트에만 극도로 절제하여 사용하십시오.
#                 -   모든 색상 조합은 WCAG AA 수준 이상의 명암비를 확보하여 접근성을 반드시 준수하도록 하십시오.
#             -   **전략적인 여백 활용(Strategic Whitespace):**
#                 -   여백은 디자인의 가장 강력한 도구 중 하나입니다. 콘텐츠 밀도를 낮추고, 각 요소를 명확히 구분하며, 사용자의 집중도를 높이고, 고급스럽고 정돈된 느낌을 극대화하기 위해 **의도적으로 매우 넉넉한 여백**을 모든 요소 주변과 섹션 사이에 배치하십시오.
#             -   **섬세한 마이크로 인터랙션(Subtle Micro-interactions):**
#                 -   사용자 경험을 향상시키고 디자인에 생동감을 미세하게 불어넣기 위해, 버튼, 링크, 카드 호버 효과 등에 부드럽고 자연스러운 CSS `transition` (예: `transition: all 0.25s cubic-bezier(0.4, 0, 0.2, 1);`)을 적용하십시오. 애니메이션은 항상 목적이 분명하고 사용자에게 방해가 되지 않아야 합니다.
#                 -   입력 필드 포커스 시 테두리 색상 변화나 미세한 그림자 효과 등을 추가할 수 있습니다.
#             -   **아이콘 플레이스홀더:** 아이콘이 필요한 위치에는 `[search icon]`, `[user settings icon]`과 같이 명확한 텍스트 플레이스홀더를 사용하거나, 가능하다면 주석으로 간단한 SVG 아이콘 코드(예: Heroicons, Feather Icons 스타일의 라인 아이콘)를 제안해주십시오.
#         4.  **스타일 가이드 주석 (Figma 참고용):** 이전 지침과 동일하게, HTML 본문 앞에 주석 형태로 이 페이지에 적용된 주요 스타일 결정사항(실제 사용된 색상값, 폰트, 주요 여백 단위 등)을 요약하여 포함해주십시오.
#         5.  **요구사항 ID 주석, 구체적인 플레이스홀더 콘텐츠, 내비게이션 링크, JavaScript 금지** 등 나머지 가이드라인은 이전 지침을 따라주십시오.

#         **최종 결과물은 어떠한 설명이나 부가적인 텍스트 없이, 순수하고 완벽한 HTML 코드 그 자체여야 합니다.** 일반적이거나 미숙해 보이는 스타일링은 절대적으로 피해주십시오. 당신은 최고 수준의 UI 디자이너입니다.
#         """

#         html_cache_key = f"html_gen_pro_designer_v1_{page_name_en}_{hash(prompt)}"

#         html_code = self._call_gpt(
#             prompt,
#             html_cache_key,
#             system_message="You are a world-class Senior UI/UX Design Lead and expert front-end developer, renowned for creating exceptionally polished, modern, minimalist, and user-centric web interfaces comparable to those produced by leading design agencies using professional tools like Figma or Stitch. You have an impeccable eye for detail, a deep understanding of visual hierarchy, advanced typography, sophisticated color theory, and interaction design principles. Your mission is to translate the given requirements into a visually stunning and functionally clear HTML/CSS mockup.",
#             temperature=0.1 # 최고 수준의 디테일과 지침 준수를 위해 매우 낮은 temperature 사용
#         )
#         return html_code

#     def generate_index_page_html(self, defined_pages_details, system_overview, project_name="소프트웨어 목업 프로젝트"):
#         # ... (변수 준비는 이전과 동일) ...
#         page_links_list_str = ""
#         for page_detail in defined_pages_details:
#             page_name_en = page_detail.get("page_name", "UnknownPage")
#             page_title_ko = page_detail.get("page_title_ko", "알 수 없는 페이지")
#             file_name = f"{sanitize_filename(page_name_en)}.html"
#             page_desc_short = page_detail.get("page_description_short", page_detail.get('page_description', 'N/A')[:50] + "...")
#             page_links_list_str += f"  <li><a href=\"{file_name}\"><strong>{page_title_ko}</strong> ({page_name_en})</a><br><small>{page_desc_short}</small></li>\n"

#         if not page_links_list_str:
#             page_links_list_str = "<li>생성된 페이지가 없습니다.</li>"
#         index_page_title = f"{project_name} - 목업 인덱스"


#         prompt = f"""
#         다음 정보를 바탕으로 이 소프트웨어 목업 프로젝트의 **최상위 인덱스 페이지(홈페이지)** HTML 코드를 생성해주십시오.
#         이 페이지는 사용자가 생성된 모든 주요 목업 페이지들을 쉽게 찾아보고 접근할 수 있도록 하는 것을 목표로 합니다.
#         **스타일 목표:** 개별 페이지들과 마찬가지로, **전문 UI 디자이너가 만든 것처럼 극도로 깔끔하고, 정교하며, 현대적인 미니멀리즘 UI 디자인**을 적용해주십시오.

#         페이지 제목 (HTML title 태그 및 화면 제목용): "{index_page_title}"
        
#         시스템 개요:
#         {system_overview}

#         생성된 주요 페이지 목록 (아래 각 항목을 클릭하면 해당 .html 파일로 이동해야 합니다.):
#         <ul class="page-link-list">
#         {page_links_list_str}
#         </ul>

#         **HTML 생성 가이드라인 (인덱스 페이지 - 전문 UI 디자이너 수준):**
#         1.  **전체 HTML 문서 구조** 및 **필수 Meta 태그**는 개별 페이지 생성 가이드라인과 동일하게 적용해주십시오.
#         2.  **시맨틱 HTML & Figma 친화적 구조**를 적용하며, 클래스명은 체계적으로(예: BEM) 작성해주십시오.
#         3.  **인라인 CSS 스타일 (최고 수준의 미니멀 & 모던 디자인):**
#             -   **모든 CSS 스타일은 HTML 코드 내 `<style>` 태그 안에 포함**해주십시오.
#             -   **주요 디자인 원칙 (전문가 수준):** 개별 페이지 생성 가이드라인에서 언급된 **정교한 레이아웃, 세련된 컴포넌트 스타일링, 고급 타이포그래피, 의도적인 색상 팔레트, 전략적인 여백 활용, 섬세한 마이크로 인터랙션** 원칙들을 이 인덱스 페이지에도 최고 수준으로 충실히 적용해주십시오.
#             -   페이지 목록은 각 항목을 명확하게 구분하고(예: 각 링크 항목을 세련된 정보 카드 형태로 표현하거나, 리스트 아이템 간 충분한 간격과 미세한 구분선 사용 등), 사용자가 쉽게 클릭하고 정보를 인지할 수 있도록 매우 높은 수준으로 스타일링 해주십시오.
#         4.  **스타일 가이드 주석 (Figma 참고용):** 개별 페이지 생성 가이드라인의 '스타일 가이드 주석' 항목을 참고하여, 이 인덱스 페이지에 적용된 주요 스타일 정보를 HTML 본문 앞에 주석으로 포함해주십시오.
#         5.  페이지 상단에는 프로젝트 이름과 함께 시스템 개요를 간략히 소개하는 섹션을 포함하여, 전체 프로젝트의 첫인상을 매우 전문적이고 세련되게 전달해주십시오. (예: 큰 타이틀, 부드러운 배경, 명확한 설명)
#         6.  그 아래에는 "생성된 목업 페이지 목록" 등과 같은 명확한 제목으로, 제공된 페이지 목록을 표시해주십시오.
#         7.  JavaScript는 포함하지 마십시오. 순수 HTML과 CSS로만 구성된 목업입니다.
#         8.  **최종 결과물은 설명이나 다른 텍스트 없이 순수 HTML 코드만이어야 합니다.**
#         """
        
#         index_cache_key = f"html_gen_index_page_pro_designer_v1_{hash(prompt)}"
        
#         html_code = self._call_gpt(
#             prompt, 
#             index_cache_key,
#             system_message="You are a world-class Senior UI/UX Design Lead, creating a stunning, minimalist, and modern index page for a web mockup project. Your work mirrors the quality of top design agencies. Respond ONLY with the raw HTML code.",
#             temperature=0.1
#         )
#         return html_code

#     def save_html_to_file(self, page_name, html_content, output_dir="mockups_output_v3"):
#         # ... (이전과 동일) ...
#         if not os.path.exists(output_dir):
#             # ...
#             return

#         safe_filename = sanitize_filename(page_name) + ".html"
#         filepath = os.path.join(output_dir, safe_filename)
        
#         try:
#             # ...
#             with open(filepath, 'w', encoding='utf-8') as f:
#                 f.write(html_content)
#             print(f"목업 파일 저장: {filepath}")
#         except Exception as e:
#             # ...
#             print(f"❌ HTML 파일 저장 중 예외 발생 ({safe_filename}): {e}")
#             import traceback
#             traceback.print_exc()



# # BEGIN: 이전 답변의 UiMockupAgent 클래스 (간략히 표시)
# class UiMockupAgent:
#     def __init__(self, requirements_file_path, openai_api_key):
#         self.requirements_file_path = requirements_file_path
#         self.openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None
#         self.loader = RequirementsLoader()
#         self.requirements_data = None
#         self.analyzer = None
#         self.planner = None
#         self.generator = None
#         self.system_overview = "N/A" # 클래스 변수로 system_overview 초기화

#     def run(self, output_dir="./generated_mockups_final_v3"):
#         print("에이전트 실행 시작...")
#         self.requirements_data = self.loader.load_from_file(self.requirements_file_path)
#         if not self.requirements_data:
#             print("요구사항 로드 실패. 에이전트 실행을 중단합니다.")
#             return None

#         if not self.openai_client:
#             print("OpenAI API 키가 설정되지 않아 GPT 기반 작업을 진행할 수 없습니다. 에이전트 실행을 중단합니다.")
#             return None

#         self.analyzer = RequirementsAnalyzer(self.requirements_data, self.openai_client)
#         self.system_overview = self.analyzer.get_system_overview() # 인스턴스 변수에 저장
#         feature_specs = self.analyzer.get_feature_specifications()

#         if not feature_specs:
#             print("기능 명세 추출 실패 또는 추출된 기능 명세가 없습니다. 에이전트 실행을 중단합니다.")
#             return None

#         print(f"\n시스템 개요: {self.system_overview}")
#         print(f"추출된/준비된 주요 기능 명세 수: {len(feature_specs)}")

#         self.planner = MockupPlanner(feature_specs, self.system_overview, self.openai_client)
#         defined_pages_with_details = self.planner.define_pages_and_allocate_features()

#         if not defined_pages_with_details or not isinstance(defined_pages_with_details, list) or not defined_pages_with_details:
#             print("페이지 정의 및 기능 할당 실패. GPT 응답을 확인하거나 MockupPlanner._get_fallback_page_plan()의 결과를 확인하십시오.")
#             if not defined_pages_with_details:
#                 print("기획된 페이지가 없습니다. 실행을 중단합니다.")
#                 return None

#         print(f"\nGPT 또는 대체 로직으로부터 기획된 페이지 수: {len(defined_pages_with_details)}")
#         for i, page_plan in enumerate(defined_pages_with_details):
#             if isinstance(page_plan, dict):
#                 print(f"  {i+1}. 페이지 영문명: {page_plan.get('page_name')}, 한글 제목: {page_plan.get('page_title_ko')}, 관련 기능 ID 수: {len(page_plan.get('included_feature_ids', []))}")
#             else:
#                 print(f"  {i+1}. 경고: 페이지 계획 형식이 잘못되었습니다: {page_plan}")

#         self.generator = HtmlGenerator(self.openai_client)
#         generated_htmls_map = {}
#         successfully_generated_page_details = []

#         for page_plan in defined_pages_with_details:
#             if not isinstance(page_plan, dict):
#                 print(f"잘못된 페이지 계획 형식으로 HTML 생성을 건너뜁니다: {page_plan}")
#                 continue

#             page_name_from_plan = page_plan.get("page_name")
#             if not page_name_from_plan:
#                 page_name_from_plan = sanitize_filename(page_plan.get("page_title_ko", f"Unknown_Page_{len(generated_htmls_map) + 1}"))
#                 print(f"경고: page_name이 없어 page_title_ko 또는 임의 이름으로 대체합니다: {page_name_from_plan}")
#                 page_plan["page_name"] = page_name_from_plan

#             print(f"\n'{page_name_from_plan}' HTML 생성 시도...")
#             html_code = self.generator.generate_html_for_page_plan(page_plan, feature_specs)

#             if html_code and "HTML 생성 중 오류 발생" not in html_code and "OpenAI 클라이언트가 설정되지 않았습니다" not in html_code:
#                 self.generator.save_html_to_file(page_name_from_plan, html_code, output_dir)
#                 generated_htmls_map[page_name_from_plan] = True
#                 successfully_generated_page_details.append(page_plan)
#             else:
#                 print(f"🔴 '{page_name_from_plan}' HTML 목업 생성 실패 또는 오류 포함된 HTML 반환.")
#                 generated_htmls_map[page_name_from_plan] = False
#                 if html_code:
#                      self.generator.save_html_to_file(f"ERROR_{page_name_from_plan}", html_code, output_dir)

#         # --- 인덱스 페이지 생성 로직 (복구 및 유지) ---
#         if successfully_generated_page_details:
#             print("\n인덱스 페이지 생성 시도...")
#             project_name_base = os.path.splitext(os.path.basename(self.requirements_file_path))[0]
#             project_name_display = project_name_base.replace("_", " ").replace("-", " ").title() + " 목업"

#             index_html_code = self.generator.generate_index_page_html(
#                 successfully_generated_page_details,
#                 self.system_overview,
#                 project_name_display
#             )
#             if index_html_code and "HTML 생성 중 오류 발생" not in index_html_code:
#                 self.generator.save_html_to_file("index", index_html_code, output_dir) # 파일명을 "index"로 지정
#                 print("🟢 인덱스 페이지(index.html) 생성 완료.")
#                 generated_htmls_map["index.html"] = True # 키를 파일명과 일치
#             else:
#                 print("🔴 인덱스 페이지 생성 실패.")
#                 generated_htmls_map["index.html"] = False
#         else:
#             print("\n성공적으로 생성된 개별 페이지가 없어 인덱스 페이지를 생성하지 않습니다.")

#         print("\n--- 최종 생성 결과 ---")
#         if any(status for status in generated_htmls_map.values()):
#             print("🟢 생성된 (또는 시도된) HTML 파일 목록:")
#             for idx, (page_key, status) in enumerate(generated_htmls_map.items(), start=1): # page_key 사용
#                 status_icon = "✅" if status else "❌"
#                 # page_key가 "index.html"일 수도 있고, page_name일 수도 있으므로, sanitize_filename 적용
#                 display_filename = page_key if page_key.endswith(".html") else f"{sanitize_filename(page_key)}.html"
#                 print(f"{idx}. {status_icon} {page_key}: {display_filename}")
#             if generated_htmls_map.get("index.html"): # 키를 "index.html"로 확인
#                  print(f"\n👉 웹 브라우저에서 '{os.path.join(output_dir, 'index.html')}' 파일을 열어 확인하세요.")
#         else:
#             print("\n🔴 생성된 유효한 HTML 목업이 없습니다.")

#         return generated_htmls_map
# # END: 이전 답변의 UiMockupAgent 클래스



In [5]:
# --- main 실행 부분 ---
if __name__ == "__main__":
    import os
    from dotenv import load_dotenv

    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

    REQUIREMENTS_FILE_PATH = "SRS_대한체육회_functional.json"
    OUTPUT_DIR = "generated_mockups_final_v3" # 출력 디렉토리명 변경

    if not OPENAI_API_KEY:
        print("❌ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")
        exit(1)

    if not os.path.exists(REQUIREMENTS_FILE_PATH):
        print(f"❌ 요구사항 파일이 존재하지 않습니다: {REQUIREMENTS_FILE_PATH}")
        exit(1)

    agent = UiMockupAgent(REQUIREMENTS_FILE_PATH, OPENAI_API_KEY)
    result = agent.run(output_dir=OUTPUT_DIR)

    if result:
        print(f"\n✅ 총 {len(result)}개의 페이지에 대한 목업 생성이 시도되었습니다. 자세한 내용은 위 로그를 확인하세요.")
    else:
        print("\n🔴 HTML 목업 생성 과정에 문제가 발생했거나 생성된 목업이 없습니다.")

에이전트 실행 시작...
요구사항 파일 'SRS_대한체육회_functional.json' 로드 성공.
시스템 개요 파악 (GPT): 주어진 소프트웨어 요구사항은 ID와 설명이 모두 'None'으로 되어 있어 구체적인 내용을 파악할 수 없습니다. 따라서, 시스템의 주요 목적, 예상되는 주요 사용자 역할(액터), ...
분석 대상 기능적 요구사항 수: 364
364개의 주요 기능 명세 추출 완료.

시스템 개요: 주어진 소프트웨어 요구사항은 ID와 설명이 모두 'None'으로 되어 있어 구체적인 내용을 파악할 수 없습니다. 따라서, 시스템의 주요 목적, 예상되는 주요 사용자 역할(액터), 그리고 시스템을 대표할 만한 이름이나 주제를 도출하기 어렵습니다.

일반적으로 소프트웨어 요구사항을 통해 시스템의 목적과 사용자 역할을 이해할 수 있지만, 이 경우에는 추가적인 정보가 필요합니다. 요구사항 문서가 완전하지 않거나 잘못된 경우, 프로젝트 팀과 협력하여 요구사항을 명확히 정의하고 문서화하는 것이 중요합니다. 이를 통해 시스템의 목표와 사용자 역할을 명확히 하고, 적절한 시스템 이름이나 주제를 선정할 수 있을 것입니다.
추출된/준비된 주요 기능 명세 수: 364
GPT에 페이지 정의 및 기능 할당 요청...
GPT 계획 요청 중 (키: page_definitions_v5_flexible)...


KeyboardInterrupt: 

In [ ]:
import os
import re
import json
from openai import OpenAI
import anthropic # Claude API 사용을 위해 추가
from dotenv import load_dotenv # .env 파일 사용을 위해 추가

# --- 유틸리티 함수 (사용자 원본 코드) ---
def sanitize_filename(name):
    """파일 이름으로 사용하기 어려운 문자를 제거하거나 대체합니다."""
    if not isinstance(name, str): # 문자열이 아닌 경우 처리
        name = str(name)
    name = re.sub(r'[<>:"/\\|?*]', '_', name) # 파일명 금지 문자 대체
    name = re.sub(r'\s+', '_', name) # 공백을 밑줄로
    return name[:100] # 파일명 길이 제한 (필요시)

# --- RequirementsLoader 클래스 (사용자 원본 코드) ---
class RequirementsLoader:
    def load_from_file(self, filepath):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            print(f"요구사항 파일 '{filepath}' 로드 성공.")
            return data
        except FileNotFoundError:
            print(f"오류: 파일 '{filepath}'를 찾을 수 없습니다.")
            return None
        except json.JSONDecodeError:
            print(f"오류: 파일 '{filepath}'가 유효한 JSON 형식이 아닙니다.")
            return None
        except Exception as e:
            print(f"파일 로드 중 예기치 않은 오류 발생: {e}")
            return None

# --- RequirementsAnalyzer 클래스 (사용자 원본 코드) ---
class RequirementsAnalyzer:
    def __init__(self, requirements_data, openai_client=None):
        self.requirements = requirements_data
        self.client = openai_client 
        self.model = "gpt-4o" 
        self.analysis_cache = {}

    def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
        if not self.client:
            print(f"OpenAI 클라이언트가 없어 GPT 분석을 건너뜁니다 ({cache_key}).")
            return None
        if cache_key in self.analysis_cache:
            return self.analysis_cache[cache_key]
        
        try:
            if hasattr(self.client, 'chat') and hasattr(self.client.chat, 'completions'): 
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": system_message},
                        {"role": "user", "content": prompt_text}
                    ],
                    temperature=0.2,
                    max_tokens=2048
                )
                result = response.choices[0].message.content.strip()
            else: 
                result = "GPT 호출 방식 오류로 분석 결과 없음"
                print(f"⚠️ RequirementsAnalyzer._call_gpt: 클라이언트 API 형식이 예상과 다릅니다. ({cache_key})")
            self.analysis_cache[cache_key] = result
            return result
        except Exception as e:
            print(f"❌ GPT 호출 실패 (키: {cache_key}) → {e}")
            return None

    def get_feature_specifications(self):
        feature_specs = []
        if not self.requirements:
            print("오류: 분석할 요구사항 데이터가 없습니다.")
            return feature_specs

        # 사용자님의 원본 필터링 로직을 그대로 유지합니다.
        target_reqs = [
            req for req in self.requirements
            if req.get("type") == "기능"
        ]
        print(f"분석 대상 기능적 요구사항 수: {len(target_reqs)}")

        for i, req in enumerate(target_reqs):
            req_id = f"FUNC-{i+1:03}"
            description = req.get("description_name", "제목 없음")
            detail = req.get("description_content", "") + "\n\n" + req.get("processing_detail", "")
            actor_guess = "사용자"

            feature_specs.append({
                "id": req_id,
                "description": description.strip(),
                "description_detailed": detail.strip(),
                "acceptance_criteria": "요구사항 내 명시 없음",
                "ui_suggestion_raw": f"'{description}' 기능을 위한 UI 구성 요소 제안",
                "actor_suggestion": actor_guess,
                "module": req.get("target_task", "미정"),
                "priority": req.get("importance", "중"),
            })
        print(f"{len(feature_specs)}개의 주요 기능 명세 추출 완료.")
        return feature_specs

    def get_system_overview(self):
        if not self.requirements: 
            return "요구사항 데이터 없음"
        
        sample_descriptions_for_overview = "\n".join([f"- ID:{req.get('id')}, 설명:{req.get('description')}" for req in self.requirements[:min(10, len(self.requirements))]])
        
        prompt = f"""다음은 소프트웨어 요구사항의 일부입니다:
        {sample_descriptions_for_overview}
        ---
        위 요구사항들을 종합하여, 이 시스템의 주요 목적은 무엇이며, 예상되는 주요 사용자 역할(액터)들은 누구인지, 그리고 이 시스템을 대표할 만한 간결한 이름이나 주제가 있다면 무엇인지 요약해주십시오.
        """
        overview = self._call_gpt(prompt, "system_overview_summary_v3", "You are a system architect summarizing project requirements.")
        
        if overview:
            print(f"시스템 개요 파악 (GPT): {overview[:100]}...")
            return overview
        else:
            first_req_desc = self.requirements[0].get("description", "상세 설명 없음")
            fallback_overview = f"총 {len(self.requirements)}개의 요구사항을 가진 시스템. 주요 목적은 '{first_req_desc}'와 관련될 것으로 보입니다."
            print(f"시스템 개요 파악 (Fallback): {fallback_overview[:100]}...")
            return fallback_overview

# --- MockupPlanner 클래스 (사용자 원본 코드) ---
class MockupPlanner:
    def __init__(self, feature_specs, system_overview, openai_client=None):
        self.feature_specs = feature_specs
        self.system_overview = system_overview
        self.client = openai_client
        self.analysis_cache = {}

    def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
        if not self.client: return None
        try:
            print(f"GPT 계획 요청 중 (키: {cache_key})...")
            response = self.client.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt_text}])
            result = response.choices[0].message.content
            self.analysis_cache[cache_key] = result
            return result
        except Exception as e:
            print(f"GPT API 호출 중 오류 발생 ({cache_key}): {e}")
            return None

    def define_pages_and_allocate_features(self):
        if not self.feature_specs:
            return self._get_fallback_page_plan()
        
        features_text_for_gpt = ""
        for spec in self.feature_specs: 
            features_text_for_gpt += f"- ID: {spec['id']}, 기능 설명: {spec['description']}\n"

        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
        # 핵심 수정: key_ui_elements_suggestion을 훨씬 더 구체적으로 요구하도록 프롬프트 변경
        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
        prompt = f"""
        시스템 개요: {self.system_overview}
        주요 기능 명세:
        {features_text_for_gpt}
        ---
        **페이지 분할 원칙 (매우 중요):**
        1.  하나의 페이지는 사용자의 **하나의 명확한 목표 또는 작업(Task)**을 해결해야 합니다.
        2.  **기능이 너무 많은 '만능 페이지'를 만들지 마십시오.** 예를 들어 '선수 관리'라는 큰 주제가 있다면, 그것을 '선수 목록 페이지', '선수 상세 정보 페이지', '선수 등록 페이지', '선수 기록 관리 페이지' 등으로 **기능을 분할하여 여러 페이지를 만드는 것을 적극적으로 고려**해주십시오.
        위 정보를 바탕으로, 이 시스템에 필요한 웹 페이지 목록을 제안하고 모든 기능을 페이지에 할당해주십시오.
        사용자 경험 흐름을 고려하여 논리적으로 그룹화하고, 다음 정보를 포함한 JSON 형식으로만 응답해주십시오.
        
        각 페이지 딕셔너리는 다음 키를 포함해야 합니다:
        1. `page_name`: 페이지의 영문 이름 (예: "Admin_User_Management")
        2. `page_title_ko`: 페이지의 한글 제목 (예: "사용자 관리")
        3. `page_description`: 페이지의 목적에 대한 간략한 설명.
        4. `target_actors`: 주요 사용자 역할 (리스트, 예: ["관리자"]) 안녕
        5. `included_feature_ids`: 이 페이지에 포함될 기능 ID 목록.
        6. `key_ui_elements_suggestion`: **(매우 중요)** 이 페이지의 기능을 구현하는 데 필요한 핵심 UI 컴포넌트들을 **구체적인 명세와 함께 제안**해주십시오. 
           **예시:** "사용자 목록을 표시하는 데이터 테이블 (컬럼: ID, 이름, 이메일, 역할, 상태), 사용자를 검색하기 위한 입력 필드와 검색 버튼, 신규 사용자 추가 버튼" 과 같이 **실제 화면에 그려야 할 요소들을 명확하고 상세하게** 작성해주십시오. 이 내용은 HTML 생성 단계에서 그대로 사용됩니다.
        """
        
        system_message = "You are an expert UI/UX designer and information architect. You must respond ONLY in valid JSON format."
        page_definitions_str = self._call_gpt(prompt, "page_definitions_v6_detailed_ui", system_message)
        
        if page_definitions_str:
            try:
                match = re.search(r'```json\s*([\s\S]*?)\s*```', page_definitions_str, re.IGNORECASE)
                json_str_cleaned = match.group(1) if match else page_definitions_str.strip()
                parsed_response = json.loads(json_str_cleaned)
                pages_list = parsed_response if isinstance(parsed_response, list) else parsed_response.get("pages")

                if isinstance(pages_list, list) and pages_list: 
                    print(f"GPT로부터 {len(pages_list)}개의 상세 페이지 계획을 성공적으로 받았습니다.")
                    return pages_list
                else:
                    print("GPT가 유효한 페이지 계획을 생성하지 못했습니다. 대체 계획을 사용합니다.")
                    return self._get_fallback_page_plan()
            except Exception as e:
                print(f"페이지 계획 파싱 오류: {e}. 대체 계획을 사용합니다.")
                return self._get_fallback_page_plan()
        else:
            print("GPT로부터 페이지 계획을 받지 못했습니다. 대체 계획을 사용합니다.")
            return self._get_fallback_page_plan()

    def _get_fallback_page_plan(self):
        print("대체 페이지 계획 사용...")
        if not self.feature_specs: return []
        main_page_features = [spec['id'] for spec in self.feature_specs]
        return [{"page_name": "Main_Application_Page_Fallback", "page_title_ko": "주요 애플리케이션 화면 (대체)", "page_description": "시스템의 모든 기능들을 제공하는 기본 페이지입니다.", "target_actors": ["사용자"], "included_feature_ids": main_page_features, "key_ui_elements_suggestion": "모든 기능에 대한 UI 요소 필요."}]
    def plan_user_main_page(self):
        """사용자 관점에서 메인 페이지에 들어갈 콘텐츠를 기획합니다."""
        print("사용자 중심 메인 페이지 콘텐츠 기획 중...")
        
        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
        # 핵심 수정: 상위 15개가 아닌, 중요도 높은 기능을 선별하여 컨텍스트로 사용
        # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
        
        # 1. 중요도가 '필수' 또는 '높음'인 기능들을 먼저 필터링합니다.
        #    (JSON 데이터에 'importance' 키와 '필수', '높음' 등의 값이 있어야 합니다)
        high_priority_specs = [
            spec for spec in self.feature_specs 
            if spec.get('priority') in ['필수', '높음', '상'] # 다양한 중요도 표현을 고려
        ]
        
        # 2. 만약 중요도 높은 기능이 없다면, 전체에서 일부를 사용합니다.
        if not high_priority_specs:
            context_specs = self.feature_specs
        else:
            context_specs = high_priority_specs

        # 3. 컨텍스트가 너무 길어지는 것을 방지하기 위해 최대 20개로 제한하되, 이제는 핵심 기능 위주입니다.
        features_list_str = "\n".join([f"- {spec['description']}" for spec in context_specs[:20]])
        print(f"메인 페이지 기획을 위해 {len(context_specs[:20])}개의 핵심 기능 컨텍스트를 사용합니다.")
        
        prompt = f"""
        당신은 '세계 최고의'의 UX 기획자입니다.
        시스템 개요: {self.system_overview}
        
        다음은 이 시스템의 **핵심 기능**들입니다:
        {features_list_str}

        ---
        위 정보를 바탕으로, **일반 사용자가 로그인했을 때 보게 될 메인 페이지(홈 화면)**에 어떤 콘텐츠가 있으면 가장 유용할지 기획해주세요.

        다음 정보를 포함한 **JSON 형식**으로만 응답해주십시오:
        1.  `page_title` (string): 페이지의 적절한 한글 제목 (예: "마이페이지", "홈")
        2.  `welcome_message` (string): 사용자 환영 메시지 (예: "{{이름}}님, 환영합니다!")
        3.  `widgets` (list of objects): 페이지에 배치할 3~5개의 위젯(콘텐츠 섹션) 목록.
            - 각 위젯 객체는 `title` (string)과 `content_idea` (string) 키를 가져야 합니다.
            - **핵심 기능 목록**을 바탕으로, 사용자가 실제로 관심을 가질 만한 내용으로 구성해주십시오.
        """
        system_message = "You are a UX planner designing a user-centric main page based on core system features. Respond ONLY in a valid JSON object."
        
        plan_str = self._call_gpt(prompt, "plan_user_main_page_v2_priority", system_message)
        
        if not plan_str:
            print("⚠️ 메인 페이지 계획 생성 실패. 기본 계획을 사용합니다.")
            return {"page_title": "메인 페이지", "welcome_message": "환영합니다!", "widgets": [{"title": "주요 기능", "content_idea": "시스템의 주요 기능 목록을 보여줍니다."}]}

        try:
            match = re.search(r'```json\s*([\s\S]*?)\s*```', plan_str, re.IGNORECASE)
            json_str = match.group(1) if match else plan_str
            return json.loads(json_str)
        except json.JSONDecodeError:
            print("⚠️ 메인 페이지 계획 JSON 파싱 실패. 기본 계획을 사용합니다.")
            return {"page_title": "메인 페이지", "welcome_message": "환영합니다!", "widgets": [{"title": "주요 기능", "content_idea": "시스템의 주요 기능 목록을 보여줍니다."}]}
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
# --- HtmlGenerator 클래스 (API 호출 엔진만 Claude로 교체) ---
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

class HtmlGenerator:
    def __init__(self, claude_client):
        self.client = claude_client
        self.analysis_cache = {}

    def _call_claude_streaming(self, prompt_text, cache_key, system_message):
        if not self.client: return ""
        if cache_key in self.analysis_cache: return self.analysis_cache[cache_key]
        
        try:
            print(f"Claude HTML 생성 요청 (키: {cache_key}, 방식: 스트리밍)...")
            with self.client.messages.stream(
                model="claude-sonnet-4-20250514",
                system=system_message,
                messages=[{"role": "user", "content": prompt_text}],
                max_tokens=4096,
                temperature=0.1
            ) as stream:
                full_response_text = "".join(stream.text_stream)

            match = re.search(r'```(html)?\s*([\s\S]*?)\s*```', full_response_text, re.IGNORECASE)
            clean_code = match.group(2).strip() if match else full_response_text.strip()
            
            if clean_code.lower().startswith('html'):
                clean_code = clean_code[4:].lstrip()

            self.analysis_cache[cache_key] = clean_code
            return clean_code

        except Exception as e:
            print(f"❌ Claude API 스트리밍 오류 ({cache_key}): {e}")
            return f"<h1>API 오류</h1><p>{e}</p>"

    # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
    # 핵심 수정: 실수로 삭제되었던 개별 페이지 생성 메서드를 복원합니다.
    # ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
    def generate_html_for_page_plan(self, page_plan_details, all_feature_specs):
        """개별 기능 페이지의 목업을 생성합니다."""
        page_title_ko = page_plan_details.get("page_title_ko", "목업 페이지")
        key_ui_elements_suggestion = page_plan_details.get("key_ui_elements_suggestion", "페이지의 주요 기능들을 표시합니다.")
        
        prompt = f"""
        **지시사항:** 당신은 세계 최고의 UI/UX 디자이너입니다. 다음 정보를 바탕으로, 실제 작동하는 것처럼 보이는 정교하고 완성도 높은 HTML 목업 페이지를 생성해주십시오. 순수 HTML과 인라인 CSS(<style> 태그)만 사용합니다.
        **스타일 목표:** 극도로 깔끔하고, 정교하며, 현대적인 미니멀리즘 UI 디자인.
        ---
        ### 페이지 정보
        - **페이지 제목:** {page_title_ko}
        - **페이지 설명:** {page_plan_details.get("page_description", "")}
        ---
        ### **페이지 핵심 구성 요소 (가장 중요! 반드시 모두 구현할 것)**
        이 페이지의 메인 콘텐츠 영역에는 다음의 구체적인 UI 컴포넌트들이 **반드시 포함**되어야 합니다. 이 지시를 무시하고 영역을 비워두지 마십시오.
        
        **{key_ui_elements_suggestion}**

        **지시 해석 예시:**
        - "사용자 목록을 표시하는 데이터 테이블"이라는 지시가 있다면, 실제 `<table>` 태그와 목업 데이터(`<tr>`, `<td>`)를 사용하여 사용자가 여러 명 있는 것처럼 표를 그려주십시오.
        - "검색 필드와 버튼"이라는 지시가 있다면, 실제 `<input type="search">`와 `<button>`을 만드십시오.
        ---
        **최종 결과물:** 다른 어떤 설명도 없이, 완성된 HTML 코드만 응답하십시오.
        """
        system_message = "You are a world-class UI/UX Design Lead. You must render all requested components. Respond ONLY with the raw HTML code."
        
        return self._call_claude_streaming(prompt, f"html_final_{page_plan_details.get('page_name')}", system_message)

    def generate_user_main_page_html(self, main_page_plan, defined_pages_details, project_name):
        """기획된 콘텐츠를 바탕으로 사용자 중심의 메인 페이지(홈)를 생성합니다."""
        page_title = main_page_plan.get("page_title", "메인 페이지")
        welcome_message = main_page_plan.get("welcome_message", "환영합니다!")
        widgets = main_page_plan.get("widgets", [])

        widgets_html = ""
        for widget in widgets:
            widgets_html += f"""
            <div class="widget-card">
                <h3 class="widget-card__title">{widget.get('title', '정보')}</h3>
                <div class="widget-card__content">
                    <p>{widget.get('content_idea', '내용이 여기에 표시됩니다.')}</p>
                </div>
            </div>
            """

        quick_links_html = ""
        for page_detail in defined_pages_details:
            file_name = f"{sanitize_filename(page_detail.get('page_name', ''))}.html"
            quick_links_html += f'<li><a href="{file_name}">{page_detail.get("page_title_ko", "링크")}</a></li>\n'
            
        prompt = f"""
        **지시사항:** 다음 기획안을 바탕으로, 사용자가 로그인했을 때 보게 될 환영 페이지를 생성해주십시오.
        **스타일 목표:** 사용자에게 친근하고, 정보가 명확하며, 현대적인 디자인.
        ---
        ### **페이지 기획안**
        1.  **페이지 전체 제목:** "{page_title}"
        2.  **페이지 상단 헤더:** 환영 메시지 "**{welcome_message}**"를 표시.
        3.  **메인 콘텐츠 (위젯 그리드):**
            - 아래 위젯들을 CSS Grid 레이아웃을 사용하여 카드 스타일로 보기 좋게 배치.
            - 각 위젯의 `content_idea`를 바탕으로, **구체적인 목업 콘텐츠(목록, 숫자, 텍스트 등)를 생성**.
            {widgets_html}
        4.  **네비게이션/사이드바 (선택 사항):**
            - 아래의 빠른 링크들을 포함한 사이드바를 왼쪽에 배치.
            - 빠른 링크 목록: <ul>{quick_links_html}</ul>
        ---
        **최종 결과물:** 설명 없이, 완성된 HTML 코드만 응답.
        """
        system_message = "You are a UI/UX designer creating a personalized, user-centric main page. Respond ONLY with the raw HTML code."
        
        return self._call_claude_streaming(prompt, f"html_user_main_page_{hash(prompt)}", system_message)

    def save_html_to_file(self, page_name, html_content, output_dir="mockups_output_final"):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        safe_filename = sanitize_filename(page_name) + ".html"
        filepath = os.path.join(output_dir, safe_filename)
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(html_content)
            print(f"목업 파일 저장: {filepath}")
        except Exception as e:
            print(f"❌ HTML 파일 저장 실패 ({filepath}): {e}")
    # save_html_to_file 메서드는 변경 없이 그대로 유지
    def save_html_to_file(self, page_name, html_content, output_dir="mockups_output_v3"):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        safe_filename = sanitize_filename(page_name) + ".html"
        filepath = os.path.join(output_dir, safe_filename)
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(html_content)
            print(f"목업 파일 저장: {filepath}")
        except Exception as e:
            print(f"❌ HTML 파일 저장 중 예외 발생 ({safe_filename}): {e}")

# --- UiMockupAgent 클래스 (Claude 클라이언트 전달하도록 수정) ---
class UiMockupAgent:
    # 1. anthropic_api_key를 추가로 받음
    def __init__(self, requirements_file_path, openai_api_key, anthropic_api_key):
        self.requirements_file_path = requirements_file_path
        self.openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None
        # 2. anthropic 클라이언트 초기화
        self.anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key) if anthropic_api_key else None
        
        self.loader = RequirementsLoader()
        self.requirements_data = None
        self.analyzer = None
        self.planner = None
        self.generator = None
        self.system_overview = "N/A"

    def run(self, output_dir="./generated_mockups_final_v3"):
        print("에이전트 실행 시작...")
        self.requirements_data = self.loader.load_from_file(self.requirements_file_path)
        if not self.requirements_data: return

        if not self.openai_client or not self.anthropic_client:
            print("OpenAI 또는 Anthropic API 키가 설정되지 않았습니다.")
            return

        self.analyzer = RequirementsAnalyzer(self.requirements_data, self.openai_client)
        self.system_overview = self.analyzer.get_system_overview()
        feature_specs = self.analyzer.get_feature_specifications()
        if not feature_specs: return

        print(f"\n시스템 개요: {self.system_overview}")
        print(f"추출된 주요 기능 명세 수: {len(feature_specs)}")

        self.planner = MockupPlanner(feature_specs, self.system_overview, self.openai_client)
        defined_pages_with_details = self.planner.define_pages_and_allocate_features()
        if not defined_pages_with_details: return

        print(f"\n기획된 페이지 수: {len(defined_pages_with_details)}")

        # 3. HtmlGenerator에 openai_client 대신 anthropic_client 전달
        self.generator = HtmlGenerator(self.anthropic_client) 
        
        # ... (이하 실행 로직은 사용자 원본과 동일하게 유지) ...
        successfully_generated_page_details = []
        for page_plan in defined_pages_with_details:
             page_name_from_plan = page_plan.get("page_name")
             if not page_name_from_plan: continue
             print(f"\n'{page_name_from_plan}' HTML 생성 시도 (with Claude)...")
             html_code = self.generator.generate_html_for_page_plan(page_plan, feature_specs)
             if html_code and "오류 발생" not in html_code:
                 self.generator.save_html_to_file(page_name_from_plan, html_code, output_dir)
                 successfully_generated_page_details.append(page_plan)
        
        if successfully_generated_page_details:
            # 4-1. 사용자 메인 페이지 콘텐츠 기획 (Planner 호출)
            print("\n--- 4. 사용자 메인 페이지 기획 단계 (OpenAI) ---")
            main_page_plan = self.planner.plan_user_main_page()
            
            # 4-2. 기획안에 따라 사용자 메인 페이지 생성 (Generator 호출)
            if main_page_plan:
                print("\n--- 5. 사용자 메인 페이지 생성 단계 (Claude) ---")
                project_name_base = os.path.splitext(os.path.basename(self.requirements_file_path))[0]
                project_name_display = project_name_base.replace("_", " ").replace("-", " ").title() + " Mockup"

                main_page_html_code = self.generator.generate_user_main_page_html(
                    main_page_plan=main_page_plan,
                    defined_pages_details=successfully_generated_page_details,
                    project_name=project_name_display
                )
                
                if main_page_html_code and "오류" not in main_page_html_code:
                    self.generator.save_html_to_file("index", main_page_html_code, output_dir)
                else:
                    print("🔴 사용자 메인 페이지 생성에 실패했습니다.")
        
        print("\n--- 최종 생성 결과 ---")
        # ... (결과 출력 로직)

# --- 실행 부분 (dotenv 사용) ---
if __name__ == '__main__':
    load_dotenv()
    
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
    
    REQUIREMENTS_FILE = "SRS_대한체육회_functional.json"

    if not OPENAI_API_KEY or not ANTHROPIC_API_KEY:
        print("!!! 중요: .env 파일에 OPENAI_API_KEY와 ANTHROPIC_API_KEY를 설정해야 합니다.")
    elif not os.path.exists(REQUIREMENTS_FILE):
        print(f"!!! 오류: 요구사항 파일 '{REQUIREMENTS_FILE}'을 찾을 수 없습니다.")
    else:
        agent = UiMockupAgent(
            requirements_file_path=REQUIREMENTS_FILE,
            openai_api_key=OPENAI_API_KEY,
            anthropic_api_key=ANTHROPIC_API_KEY
        )
        agent.run()

에이전트 실행 시작...
요구사항 파일 'SRS_대한체육회_functional.json' 로드 성공.
시스템 개요 파악 (GPT): 주어진 소프트웨어 요구사항 목록은 모두 'ID'와 '설명'이 'None'으로 되어 있어, 구체적인 정보를 제공하지 않습니다. 따라서, 이 목록만으로는 시스템의 주요 목적, 예상되는...
분석 대상 기능적 요구사항 수: 364
364개의 주요 기능 명세 추출 완료.

시스템 개요: 주어진 소프트웨어 요구사항 목록은 모두 'ID'와 '설명'이 'None'으로 되어 있어, 구체적인 정보를 제공하지 않습니다. 따라서, 이 목록만으로는 시스템의 주요 목적, 예상되는 주요 사용자 역할(액터), 또는 시스템을 대표할 만한 이름이나 주제를 도출하기 어렵습니다.

일반적으로 소프트웨어 요구사항 문서에는 시스템의 기능적 요구사항과 비기능적 요구사항이 명확히 기술되어 있어야 하며, 이를 통해 시스템의 목적과 사용자 역할을 파악할 수 있습니다. 추가적인 정보나 구체적인 요구사항이 제공된다면, 보다 정확한 분석과 요약이 가능할 것입니다. 

추가적인 세부사항을 제공해 주시면, 더 나은 요약을 제공할 수 있도록 하겠습니다.
추출된 주요 기능 명세 수: 364
GPT 계획 요청 중 (키: page_definitions_v6_detailed_ui)...
GPT로부터 10개의 상세 페이지 계획을 성공적으로 받았습니다.

기획된 페이지 수: 10

'Athlete_List' HTML 생성 시도 (with Claude)...
Claude HTML 생성 요청 (키: html_final_Athlete_List, 방식: 스트리밍)...
목업 파일 저장: ./generated_mockups_final_v3\Athlete_List.html

'Athlete_Details' HTML 생성 시도 (with Claude)...
Claude HTML 생성 요청 (키: html_final_Athlete_Details, 방식: 스트리밍)...
목업 파일 저장: ./generated_